In [88]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
from PIL import Image
from tqdm import tqdm

In [19]:
labels_path = "/home/filiptkac/Downloads/train.csv"
images_path = "/home/filiptkac/Downloads/abd-trauma-dataset"

In [15]:
df = pd.read_csv(labels_path)
df.head(5)

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1
1,10005,1,0,1,0,1,0,0,1,0,0,1,0,0,0
2,10007,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3,10026,1,0,1,0,1,0,0,1,0,0,1,0,0,0
4,10051,1,0,1,0,1,0,0,1,0,0,0,1,0,1


In [24]:
patients = os.listdir(images_path)

min_images = None
max_images = None
patients_too_few_images = []

for patient in patients:
    files = os.listdir(f"{images_path}/{patient}")
    files_amount = len(files)
    if min_images is None or files_amount < min_images:
        min_images = files_amount
    if max_images is None or files_amount > max_images:
        max_images = files_amount
    if files_amount < 100:
        patients_too_few_images.append(patient)

print(f"minimum number of images in CT: {min_images}, maximum number of images in CT: {max_images}")
print(f"number of CT scans with less than hundred images: {len(patients_too_few_images)}")

minimum number of images in CT: 46, maximum number of images in CT: 1727
number of CT scans with less than hundred images: 261


In [90]:
num_of_channels = 100
patient_images = np.zeros((len(patients) - len (patients_too_few_images), num_of_channels, 128, 128))

for idx, patient in enumerate(tqdm(patients)):
    if patient in patients_too_few_images:
        continue

    files = os.listdir(f"{images_path}/{patient}")
    # take 100 images, so they're evenly spaced from the original images
    file_indices = np.linspace(0, len(files) - 1, num_of_channels, dtype=int)
    files = np.array(files)[file_indices]

    images_per_patient = []
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            img = Image.open(os.path.join(images_path, patient, file)).convert("L")
            img_array = np.array(img) / 255.0
            images_per_patient.append(img_array)

    patient_images[idx, :, :, :] = np.array(images_per_patient)
    
print(patient_images.shape)

MemoryError: Unable to allocate 35.2 GiB for an array with shape (2886, 100, 128, 128) and data type float64

In [28]:
test_linspace = np.linspace(0, 150, 100)
print(np.uint32(test_linspace))

[  0   1   3   4   6   7   9  10  12  13  15  16  18  19  21  22  24  25
  27  28  30  31  33  34  36  37  39  40  42  43  45  46  48  50  51  53
  54  56  57  59  60  62  63  65  66  68  69  71  72  74  75  77  78  80
  81  83  84  86  87  89  90  92  93  95  96  98 100 101 103 104 106 107
 109 110 112 113 115 116 118 119 121 122 124 125 127 128 130 131 133 134
 136 137 139 140 142 143 145 146 148 150]
